In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem import SaltRemover
from networkx.algorithms.clique import find_cliques
from networkx import to_numpy_matrix
from func_timeout import func_timeout, FunctionTimedOut
from mmpa.mmp import MMP

In [3]:
from tqdm import tqdm
tqdm.pandas()

### import all activity values for compounds within 40% similarity of adenaline

In [4]:
df = pd.read_csv('/home/daniel/Downloads/DOWNLOAD-DMAOS_6IYWiRfDdsg31VJek5bJXL5Y2ndfPnJhrbTB0=.csv.gz', sep=';', compression='gzip')

### most common molecules

In [5]:
df['Molecule Name'].value_counts()[0:10]

ISOPROTERENOL                  1894
EPINEPHRINE                    1025
NOREPINEPHRINE                  851
PHENYLEPHRINE                   566
ALBUTEROL                       493
RACEPINEPHRINE                  449
LEVISOPRENALINE                 143
ISOPROTERENOL HYDROCHLORIDE      96
NOREPINEPHRINE BITARTRATE        86
EPINEPHRINE BITARTRATE           79
Name: Molecule Name, dtype: int64

### most common targets

In [6]:
df['Target Name'].value_counts()[0:10]

Rattus norvegicus             3191
Unchecked                      665
Beta-2 adrenergic receptor     363
Beta-1 adrenergic receptor     132
Hepatotoxicity                 130
Canis familiaris               111
NON-PROTEIN TARGET             109
ADMET                           79
Beta-3 adrenergic receptor      65
Cavia porcellus                 64
Name: Target Name, dtype: int64

### filter beta2 measurements

In [7]:
df_beta2 = df[df['Target Name']=='Beta-2 adrenergic receptor']
df_beta2.Smiles.unique().size

56

### strip salts

In [8]:
def strip_salts(smiles, remover):
    mol = Chem.MolFromSmiles(smiles)
    res, deleted = remover.StripMolWithDeleted(mol)
    res = Chem.MolToSmiles(res)
    deleted = [Chem.MolToSmarts(mol) for mol in deleted]
    return pd.Series({'Stripped':res, 'Salts':deleted})

In [9]:
remover = SaltRemover.SaltRemover()
df_beta2 = df_beta2.join(df_beta2.Smiles.apply(strip_salts, remover=remover))
df_beta2.Stripped.unique().size

47

### create cartesian product of molecules test in the same assay

In [10]:
df_beta2_pairs = pd.merge(df_beta2, df_beta2, on='Assay ChEMBL ID')
df_beta2_pairs = df_beta2_pairs[['Stripped_x', 'Stripped_y']].drop_duplicates()
df_beta2_pairs

,Stripped_x,Stripped_y
0,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1
1,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CCC(NC(C)C)C(O)c1ccc(O)c(O)c1
2,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NC[C@H](O)c1ccc(O)c(O)c1
3,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1
4,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(O)c(O)c1
...,...,...
1938,CC(C)NCC(O)c1ccc(O)c(NS(C)(=O)=O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1
1946,CC(C)(C)NCC(O)c1ccc(O)c(NC(N)=O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1
1954,CC(C)NCC(O)c1ccc(O)cc1,COC(CNC(C)C)c1ccc(O)c(O)c1
1962,CC(C)(C)NCC(O)c1ccc(O)c(O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1


### identify pairs

In [11]:
def apply_mmpa(prospective_pair):
    
    # define input molecules
    mol1 = Chem.MolFromSmiles(prospective_pair.Stripped_x)
    mol2 = Chem.MolFromSmiles(prospective_pair.Stripped_y)

    # prepare potential atom-atom mappings and create correspondence graph
    mmp = MMP()
    mmp.setMol1(mol1)
    mmp.setMol2(mol2)
    mmp.createCorrespondence(penalty=3.0)
    
    # score the cliques and isolate RECS 
    def find_cliques_no_args():
        return list(find_cliques(mmp))
    try:
        cliques = func_timeout(60, find_cliques_no_args)
    except FunctionTimedOut:
        print("find_cliques could not complete within 60 seconds, hence terminated.")
        return prospective_pair
    mmp.scoreCliques(cliques) 
    mmp.eliminateMCS()
    
    # append frags to output
    prospective_pair['Frag_x'] = mmp.getFragmentA()
    prospective_pair['Frag_y'] = mmp.getFragmentB()
    
    # create reaction
    smirks = '{}>>{}'.format(Chem.MolToSmarts(Chem.AddHs(mmp.frag1)), Chem.MolToSmarts(Chem.AddHs(mmp.frag2)))
    rxn = Chem.rdChemReactions.ReactionFromSmarts(smirks)
    if rxn.GetNumReactantTemplates() != 1 or rxn.GetNumProductTemplates() != 1: return prospective_pair
    
    # return
    prospective_pair['SMIRKS'] = smirks
    return prospective_pair

In [12]:
df_beta2_pairs = df_beta2_pairs.progress_apply(apply_mmpa, axis=1)
df_beta2_pairs = df_beta2_pairs[~pd.isna(df_beta2_pairs.SMIRKS)]
df_beta2_pairs

100%|██████████| 551/551 [00:41<00:00, 13.16it/s]


,Frag_x,Frag_y,SMIRKS,Stripped_x,Stripped_y
1,[H][C@@](O)(CNC(C)C)c1cccc(O)c1,CCC(NC(C)C)C(O)c1cccc(O)c1,[#6:13](-[#6:5](-[#6:12](-[H])(-[H])-[H])(-[#7...,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CCC(NC(C)C)C(O)c1ccc(O)c(O)c1
2,CC(C)NCC(O)c1cccc(O)c1,CC(C)NCC(O)c1cccc(O)c1,[#6:10](-[#6:5](-[#6:11](-[H])(-[H])-[H])(-[#7...,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NC[C@H](O)c1ccc(O)c(O)c1
3,[H][C@@](O)(CNC(C)C)c1ccc(O)c(O)c1,CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1,[#6](-[#6](-[#6](-[H])(-[H])-[H])(-[#7:1](-[#6...,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1
4,[H][C@@](O)(CNC(C)C)c1cccc(O)c1,CNCC(O)c1cccc(O)c1,[#6](-[#6](-[#6](-[H])(-[H])-[H])(-[#7:6](-[#6...,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(O)c(O)c1
5,[H][C@@](O)(CNC(C)C)c1cccc(O)c1,CC(C)NCC(O)c1cccc(O)c1,[#6:8](-[#6:3](-[#6:9](-[H])(-[H])-[H])(-[#7:5...,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NCC(O)c1ccc(O)c(O)c1
...,...,...,...,...,...
1938,CNCC(O)c1ccc(O)c(NS(C)(=O)=O)c1,CNCC(OC)c1ccc(O)c(O)c1,[#6:9](-[#7:10](-[#6:11](-[#6:6](-[#8]-[H])(-[...,CC(C)NCC(O)c1ccc(O)c(NS(C)(=O)=O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1
1946,CC(C)(C)NCC(O)c1ccc(O)c(NC(N)=O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1,[#6](-[#6](-[#6:11](-[H])(-[H])-[H])(-[#6:12](...,CC(C)(C)NCC(O)c1ccc(O)c(NC(N)=O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1
1954,CNCC(O)c1ccc(O)cc1,CNCC(OC)c1ccc(O)c(O)c1,[#6:5](-[#7:6](-[#6:7](-[#6:1](-[#8]-[H])(-[#6...,CC(C)NCC(O)c1ccc(O)cc1,COC(CNC(C)C)c1ccc(O)c(O)c1
1962,ccc(cc)C(O)CNC(C)(C)C,ccc(cc)C(CNC(C)C)OC,[#6](-[#6](-[#6:12](-[H])(-[H])-[H])(-[#6:13](...,CC(C)(C)NCC(O)c1ccc(O)c(O)c1,COC(CNC(C)C)c1ccc(O)c(O)c1


### quality control, demonstrate x -> y using reaction

In [13]:
def apply_reactions(reaction):
    
    # create reaction
    rxn = Chem.rdChemReactions.ReactionFromSmarts(reaction.SMIRKS)

    # enumerate products
    reaction['Products'] = rxn.RunReactants((Chem.AddHs(Chem.MolFromSmiles(reaction.Stripped_x)),))
    return reaction

In [14]:
df_beta2_products = df_beta2_pairs.progress_apply(apply_reactions, axis=1)
df_beta2_products.sample(3)

100%|██████████| 490/490 [00:11<00:00, 43.24it/s]


,Frag_x,Frag_y,SMIRKS,Stripped_x,Stripped_y,Products
98,[H][C@](O)(CNC)c1ccc(O)c(O)c1,CNCC(O)c1ccc(O)c(O)c1F,[#6:8](-[#7:4](-[#6:6](-[#6@](-[#8:11]-[H])(-[...,CNC[C@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(O)c(O)c1F,((<rdkit.Chem.rdchem.Mol object at 0x7fd11ab73...
413,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,CC(C)NCC(O)c1ccc(O)cc1,[#6:9](-[#6](-[#6](-[H])(-[H])-[H])(-[#6:8](-[...,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,CC(C)NCC(O)c1ccc(O)cc1,((<rdkit.Chem.rdchem.Mol object at 0x7fd11a3ce...
97,NCC(O)c1ccc(O)c(O)c1,NCC(O)c1ccc(O)c(O)c1F,[#7:4](-[#6:7](-[#6@:11](-[#8:13]-[H])-[#6:6]1...,CNC[C@H](O)c1ccc(O)c(O)c1,CNC[C@H](O)c1ccc(O)c(O)c1F,((<rdkit.Chem.rdchem.Mol object at 0x7fd11ab73...


In [15]:
len(df_beta2_products.index)

490

In [16]:
def products_to_list(productset):
    
    # given produces exist
    productlist = []
    for product in productset:
        productlist.append('.'.join([Chem.MolToSmiles(Chem.RemoveHs(productpart)) for productpart in product]))
    return list(set(productlist))

df_beta2_products['ProductList'] = df_beta2_products.Products.apply(lambda x: products_to_list(x) if pd.notna(x) else list())

In [17]:
df_beta2_products.sample(3)

,Frag_x,Frag_y,SMIRKS,Stripped_x,Stripped_y,Products,ProductList
26,CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1,CC(C)NCC(O)c1ccc(O)c(O)c1,[#6](-[#7:1](-[#6:2](-[#6:3](-[#8:4]-[H])(-[#6...,CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1,CC(C)NCC(O)c1ccc(O)c(O)c1,((<rdkit.Chem.rdchem.Mol object at 0x7fd11ab94...,[CC(C)NCC(O)c1ccc(O)c(O)c1]
416,Cc1ccc(O)c(CO)c1,Cc1ccc(O)c(N)c1,[#6:2](-[#6:6]1:[#6:11](:[#6:9](:[#6:8](-[#8:7...,CC(C)(C)NCC(O)c1ccc(O)c(CO)c1,CC(C)(C)NCC(O)c1ccc(O)c(N)c1,((<rdkit.Chem.rdchem.Mol object at 0x7fd11a3f0...,[CC(C)(C)NCC(O)c1ccc(O)c(N)c1]
332,CNc1cc(C)ccc1O,Cc1ccc(O)c(N)c1,[#6](-[#7](-[#6:10]1:[#6:9](:[#6:6](-[#6:2]-[H...,CNc1cc(C(O)CNC(C)(C)C)ccc1O,CC(C)(C)NCC(O)c1ccc(O)c(N)c1,((<rdkit.Chem.rdchem.Mol object at 0x7fd11a798...,[CC(C)(C)NCC(O)c1ccc(O)c(N)c1]


In [18]:
df_beta2_products.apply(lambda x: x.Stripped_y not in x.ProductList, axis=1).sum()

0

In [19]:
df_beta2_products.ProductList.apply(lambda x: len(x) > 1).sum()

0